In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

/var/folders/h7/fr7g30_x5x12d68ww_s6w2_m0000gn/T/ipykernel_91730/435009928.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
curr_folder = 'real2'
csv_path = f'/Users/melloo21/Desktop/NUS Items/CS4220/cs4220_projects/data/raw_csv/{curr_folder}_final.csv'
bed_path = f'/Users/melloo21/Desktop/NUS Items/CS4220/cs4220_projects/data/{curr_folder}'
NUM_TOP = 2
algo_dict = {
    'mutect2': 'FILTER_Mutect2',
    'varscan': 'FILTER_Varscan',
    'vardict': 'FILTER_Vardict',
    'freebayes': 'FILTER_Freebayes'
}

In [3]:
df = pd.read_csv(csv_path, delimiter='\t')


/var/folders/h7/fr7g30_x5x12d68ww_s6w2_m0000gn/T/ipykernel_91730/3912418589.py:1: DtypeWarning: Columns (23,24,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, delimiter='\t')


In [4]:
def get_f1(df, pred_col, true_col):
    y_pred = df[pred_col].to_numpy()
    y_true = df[true_col].to_numpy()
    return f1_score(y_true, y_pred)
    
def naive_algo_1(df):
    df['count_num_true'] = df[algo_dict.values()].sum(axis=1)
    df['pred_by_at_least_2'] = df.apply(lambda row: True if row['count_num_true'] > 1 else False, axis=1)
    df_to_bed = df[df['pred_by_at_least_2'] == True].reset_index(drop=True)
    df_to_bed = df_to_bed[['Chr', 'START_POS_REF', 'END_POS_REF']]
    df_to_bed.to_csv(f'{bed_path}/{curr_folder}_naive1.bed', sep='\t', header=False, index=False)
    print("\nF1 output from at least 2 algorithms:" )
    print(get_f1(df, 'pred_by_at_least_2', 'is_snv'))

def naive_algo_2(df):
    f1_score_dict = {}
    for algo, col in algo_dict.items():
        f1_score_dict[col] = get_f1(df, col, 'is_snv')

    print("F1 output from individual algos:")
    print(f1_score_dict)
    top_cols = dict(sorted(f1_score_dict.items(), key = lambda x: x[1], reverse = True)[:NUM_TOP])
    df['pred_by_top_2'] = df.apply(lambda row: all(row[col] for col in top_cols), axis=1)
    df_to_bed = df[df['pred_by_top_2'] == True].reset_index(drop=True)
    df_to_bed = df_to_bed[['Chr', 'START_POS_REF', 'END_POS_REF']]
    df_to_bed.to_csv(f'{bed_path}/{curr_folder}_naive2.bed', sep='\t', header=False, index=False)
    print(f"\nF1 output from top 2 algorithms: {top_cols.keys()}" )
    print(get_f1(df, 'pred_by_top_2', 'is_snv'))


In [5]:
naive_algo_2(df)

F1 output from individual algos:
{'FILTER_Mutect2': 0.2416551724137931, 'FILTER_Varscan': 0.017048188629958067, 'FILTER_Vardict': 0.13489527866524673, 'FILTER_Freebayes': 0.21490467937608318}

F1 output from top 2 algorithms: dict_keys(['FILTER_Mutect2', 'FILTER_Freebayes'])
0.7969094922737306


In [6]:
naive_algo_1(df)


F1 output from at least 2 algorithms:
0.3031496062992126
